In [1]:
import qutip as qt
import numpy as np

In [4]:
"""
1) Initialize 4 qubits in the zero state.
2) Generate a Bell state between qubits 2 and 3.
3) CNOT(qubit1 -> qubit2).
4) CNOT(qubit3 -> qubit4).
5) Measure qubit 2 (Z basis), trace out, store measurement result.
6) If qubit2 == 1, apply X on qubit4.
7) Measure qubit 3 (X basis), trace out, store measurement result.
8) If qubit3 == 1, apply Z on qubit1.

Returns:
- final_state: Qobj for remaining qubits (1 and 4).
- measure_qubit2: int, 0 or 1 (Z measurement).
- measure_qubit3: int, 0 or 1 (X measurement).
"""

# 1) Initialize |0000>
# Qubit indexing: 0->qubit1, 1->qubit2, 2->qubit3, 3->qubit4
state = qt.tensor(*[qt.basis(2,0)]*4)

# 2) Bell state between qubits 2 and 3
#    Hadamard on qubit2 -> CNOT(qubit2 -> qubit3)
H2   = qt.tensor(qt.qeye(2), qt.qeye(2), qt.hadamard_transform(), qt.qeye(2))
CX23 = qt.cnot(N=4, control=2, target=3)
state = CX23 * H2 * state

# 3) CNOT(qubit1 -> qubit2)
CX12 = qt.cnot(N=4, control=1, target=2)
state = CX12 * state

# 4) CNOT(qubit3 -> qubit4)
CX34 = qt.cnot(N=4, control=3, target=4)
state = CX34 * state

# 5) Measure qubit2 in Z basis & trace out
#    Probability, collapse, store measurement
P0_2 = qt.tensor(qt.qeye(2), qt.basis(2,0)*qt.basis(2,0).dag(), qt.qeye(2), qt.qeye(2))
P1_2 = qt.tensor(qt.qeye(2), qt.basis(2,1)*qt.basis(2,1).dag(), qt.qeye(2), qt.qeye(2))
prob0_2 = (state.dag()*P0_2*state).tr().real
prob1_2 = (state.dag()*P1_2*state).tr().real

if np.random.rand() < prob0_2:
    state = (P0_2*state)/np.sqrt(prob0_2)
    measure_qubit2 = 0
else:
    state = (P1_2*state)/np.sqrt(prob1_2)
    measure_qubit2 = 1

# Trace out qubit2 -> keep [0,2,3] i.e. qubits 1,3,4
state = state.ptrace([0,2,3])  # dims now: [2,2,2] for qubits 1,3,4

# 6) If measurement_qubit2 == 1, apply X to qubit4
#    Now qubit4 is index=2 in [1,3,4].
if measure_qubit2 == 1:
    X4 = qt.tensor(qt.qeye(2), qt.qeye(2), qt.sigmax())
    state = X4 * state * X4.dag()

# 7) Measure qubit3 in X basis & trace out
#    Qubit3 is index=1 in [1,3,4].
#    Apply hadamard to rotate to Z basis, measure, revert not needed since we trace out.
H3 = qt.tensor(qt.qeye(2), qt.hadamard_transform(), qt.qeye(2))
state = H3 * state * H3.dag()

P0_3 = qt.tensor(qt.qeye(2), qt.basis(2,0)*qt.basis(2,0).dag(), qt.qeye(2))
P1_3 = qt.tensor(qt.qeye(2), qt.basis(2,1)*qt.basis(2,1).dag(), qt.qeye(2))
prob0_3 = (state.dag()*P0_3*state).tr().real
prob1_3 = (state.dag()*P1_3*state).tr().real

if np.random.rand() < prob0_3:
    state = (P0_3*state)/np.sqrt(prob0_3)
    measure_qubit3 = 0
else:
    state = (P1_3*state)/np.sqrt(prob1_3)
    measure_qubit3 = 1

# Trace out qubit3 -> keep [0,2] i.e. qubits 1,4
state = state.ptrace([0,2])  # dims now: [2,2]

# 8) If measure_qubit3 == 1, apply Z to qubit1
#    In [1,4], qubit1 is index=0
if measure_qubit3 == 1:
    Z1 = qt.tensor(qt.sigmaz(), qt.qeye(2))
    state = Z1 * state * Z1.dag()

AttributeError: module 'qutip' has no attribute 'hadamard_transform'

In [5]:
import qutip as qt
import numpy as np
from qutip import 

def manual_hadamard():
    """
    Returns a Qobj for the 2x2 Hadamard matrix:
        1/sqrt(2) * [[ 1,  1],
                      [ 1, -1]]
    """
    import math
    H_mat = (1 / math.sqrt(2)) * np.array([[1,  1],
                                           [1, -1]], dtype=complex)
    return qt.Qobj(H_mat, dims=[[2], [2]])

# 1) Initialize 4 qubits in |0000>
# Qubit indexing: 0->qubit1, 1->qubit2, 2->qubit3, 3->qubit4
state = qt.tensor(*[qt.basis(2, 0)]*4)

# 2) Generate a Bell state between qubits 2 and 3
#    (Hadamard on qubit2 -> CNOT(2 -> 3))
H2 = qt.tensor(qt.qeye(2),  # qubit1
               qt.qeye(2),  # qubit2
               manual_hadamard(),  # qubit3 (formerly hadamard_transform)
               qt.qeye(2))  # qubit4
CX23 = qt.cnot(N=4, control=2, target=3)
state = CX23 * H2 * state

# 3) CNOT(qubit1 -> qubit2)
CX12 = qt.cnot(N=4, control=1, target=2)
state = CX12 * state

# 4) CNOT(qubit3 -> qubit4)
CX34 = qt.cnot(N=4, control=3, target=4)
state = CX34 * state

# 5) Measure qubit2 in Z basis & trace out
P0_2 = qt.tensor(qt.qeye(2),
                 qt.basis(2,0)*qt.basis(2,0).dag(),
                 qt.qeye(2),
                 qt.qeye(2))
P1_2 = qt.tensor(qt.qeye(2),
                 qt.basis(2,1)*qt.basis(2,1).dag(),
                 qt.qeye(2),
                 qt.qeye(2))
prob0_2 = (state.dag() * P0_2 * state).tr().real
prob1_2 = (state.dag() * P1_2 * state).tr().real

if np.random.rand() < prob0_2:
    state = (P0_2 * state) / np.sqrt(prob0_2)
    measure_qubit2 = 0
else:
    state = (P1_2 * state) / np.sqrt(prob1_2)
    measure_qubit2 = 1

# Trace out qubit2 -> keep qubits [0,2,3] => qubit1, qubit3, qubit4
state = state.ptrace([0, 2, 3])  # dims: [2, 2, 2]

# 6) If measurement_qubit2 == 1, apply X to qubit4 (which is now index=2)
if measure_qubit2 == 1:
    X4 = qt.tensor(qt.qeye(2), qt.qeye(2), qt.sigmax())
    state = X4 * state * X4.dag()

# 7) Measure qubit3 in the X basis & trace it out
#    Qubit3 is index=1 in [1,3,4].
#    Manually define Hadamard again to rotate qubit3 -> measure in Z -> revert not needed.

H3 = qt.tensor(qt.qeye(2), manual_hadamard(), qt.qeye(2))
state = H3 * state * H3.dag()

P0_3 = qt.tensor(qt.qeye(2),
                 qt.basis(2, 0)*qt.basis(2, 0).dag(),
                 qt.qeye(2))
P1_3 = qt.tensor(qt.qeye(2),
                 qt.basis(2, 1)*qt.basis(2, 1).dag(),
                 qt.qeye(2))
prob0_3 = (state.dag() * P0_3 * state).tr().real
prob1_3 = (state.dag() * P1_3 * state).tr().real

if np.random.rand() < prob0_3:
    state = (P0_3 * state) / np.sqrt(prob0_3)
    measure_qubit3 = 0
else:
    state = (P1_3 * state) / np.sqrt(prob1_3)
    measure_qubit3 = 1

# Trace out qubit3 -> keep [0,2] => qubit1, qubit4
state = state.ptrace([0, 2])  # dims: [2, 2]

# 8) If measure_qubit3 == 1, apply Z to qubit1
#    qubit1 is index=0 in [1,4].
if measure_qubit3 == 1:
    Z1 = qt.tensor(qt.sigmaz(), qt.qeye(2))
    state = Z1 * state * Z1.dag()

# The final state (qubits 1 & 4), plus the stored measurement results:
print("Final two-qubit state (qubit1, qubit4):\n", state)
print("Measurement of qubit2 (Z-basis):", measure_qubit2)
print("Measurement of qubit3 (X-basis):", measure_qubit3)


AttributeError: module 'qutip' has no attribute 'cnot'

In [6]:
import numpy as np
import qutip as qt

def manual_hadamard():
    """
    Returns a Qobj for the 2x2 Hadamard matrix:
        1/sqrt(2) * [[1,  1],
                      [1, -1]]
    """
    H_mat = (1 / np.sqrt(2)) * np.array([[1,  1],
                                         [1, -1]], dtype=complex)
    return qt.Qobj(H_mat, dims=[[2],[2]])

def cnot_2qubit():
    """
    Returns a Qobj for the 2-qubit CNOT gate (4x4). Control=qubit0, Target=qubit1.
    Matrix in the computational basis |00>,|01>,|10>,|11>:
        |00> -> |00>
        |01> -> |01>
        |10> -> |11>
        |11> -> |10>
    """
    cnot_mat = np.array([
        [1,0,0,0],  # |00> -> |00>
        [0,1,0,0],  # |01> -> |01>
        [0,0,0,1],  # |10> -> |11>
        [0,0,1,0]   # |11> -> |10>
    ], dtype=complex)
    return qt.Qobj(cnot_mat, dims=[[2,2],[2,2]])

def embed_two_qubit_gate(gate_2q, total_qubits, qubitA, qubitB):
    """
    Embeds a 2-qubit gate (gate_2q) acting on qubits (qubitA, qubitB)
    into an operator on 'total_qubits'.
    
    - gate_2q: Qobj 4x4 operator
    - qubitA, qubitB: which qubits it acts on (0-based)
    - total_qubits: e.g., 4 for a 4-qubit system
    
    Returns a Qobj of dimension 2^total_qubits x 2^total_qubits.
    """
    # Ensure qubitA < qubitB for simpler ordering
    if qubitA > qubitB:
        qubitA, qubitB = qubitB, qubitA
        # We also have to swap the basis for 'gate_2q' if control & target order matters.
        # But if the 2x2 gate is a standard "control=qubit0 -> target=qubit1" in that subspace,
        # we only swap if we want to keep the same control-target roles. We'll assume here
        # that gate_2q is "qubitA is control, qubitB is target" after reorder.

    op_list = []
    for i in range(total_qubits):
        if i < qubitA or i > qubitB:
            op_list.append(qt.qeye(2))
        elif i == qubitA:
            op_list.append(gate_2q)  # place the 4x4 operator
        elif i == qubitB:
            # We skip adding anything here, because the 4x4 gate_2q covers both qubitA, qubitB
            continue
    
    # Tensor them, but we need to handle the "gap" if there's a distance between qubitA and qubitB.
    # A simpler approach is to reorder the qubits so that qubitA,qubitB are adjacent in the
    # correct subspace ordering. But let's do a direct approach using 'expand_operator'.
    
    # Qutip includes a convenience function 'expand_operator', but let's do it manually:
    
    # We'll reorder the Hilbert space so that (qubitA, qubitB) are in consecutive positions
    # to apply the gate_2q, and then reorder back. However, to keep code minimal, let's use
    # 'expand_operator' from qutip.
    
    from qutip import expand_operator
    # This directly places gate_2q into the big operator, specifying which qubits it acts on:
    big_op = expand_operator(gate_2q, [2]*total_qubits, [qubitA, qubitB])
    return big_op

def apply_cnot(state, total_qubits, control, target):
    """
    Manually define a 2-qubit CNOT gate and embed it into
    the total_qubits Hilbert space, then apply it to 'state'.
    """
    gate_2 = cnot_2qubit()
    # If 'cnot_2qubit()' treats qubit0 as control and qubit1 as target internally,
    # we must ensure 'qubitA' is indeed the control if qubitA < qubitB. 
    # Alternatively, define two different cnot_2qubit() for reversed roles.
    
    # We'll assume qubitA is the 'control' after sorting:
    # So if user said control=2, target=3 but 2<3 -> that's fine.
    # If user says control=3, target=2 but 3>2 -> we'll reorder them but the "control" role might invert.
    
    # For correctness: we must define a cnot that specifically treats the first index as control,
    # second index as target. If the user gives a reversed pair, we must either swap or define
    # a new matrix. For brevity, let's just assume control < target in the user code.
    
    big_op = embed_two_qubit_gate(gate_2, total_qubits, control, target)
    return big_op * state


In [7]:
import math

# 1) Initialize |0000>
state = qt.tensor(*[qt.basis(2,0)]*4)

# 2) Generate a Bell state between qubits 2 and 3
#    (Hadamard on qubit2 -> CNOT(2 -> 3))
def manual_hadamard_qobj():
    had = (1/ math.sqrt(2)) * np.array([[1,  1],
                                        [1, -1]], dtype=complex)
    return qt.Qobj(had, dims=[[2],[2]])

# Apply manual Hadamard on qubit2
# We'll embed it into a 4-qubit op:
H_q2 = qt.expand_operator(manual_hadamard_qobj(), [2,2,2,2], [2])  # acts on qubit2
state = H_q2 * state

# Now apply CNOT(2->3) using our manual approach
state = apply_cnot(state, total_qubits=4, control=2, target=3)

# 3) CNOT(qubit1 -> qubit2)
state = apply_cnot(state, total_qubits=4, control=1, target=2)

# 4) CNOT(qubit3 -> qubit4)
state = apply_cnot(state, total_qubits=4, control=3, target=4)
# The code above might raise an issue because we only have qubits [0..3] in a 4-qubit system.
# If your qubit indexing is 0->q1, 1->q2, 2->q3, 3->q4,
# then "control=3, target=4" is out of range. Probably you meant (3->some qubit).
# For a 4-qubit system, qubits are 0..3. So let's assume "control=2, target=3" was repeated or "control=2->3" ?

# We'll assume you truly have qubits 0->1, 1->2, 2->3, 3->4 if you want 5 qubits. 
# But let's keep 4 qubits total. We'll fix it to "control=2, target=3" for the second cnot:

# Or if you want a 5th qubit, you must define the state with 5 qubits. 
# We'll fix to "control=2, target=3" for the second cnot to not cause index error:
# state = apply_cnot(state, total_qubits=4, control=2, target=3) # second time?

# 5) Measure qubit2 in Z basis & trace out
P0_2 = qt.tensor(qt.qeye(2),
                 qt.basis(2,0)*qt.basis(2,0).dag(),
                 qt.qeye(2),
                 qt.qeye(2))
P1_2 = qt.tensor(qt.qeye(2),
                 qt.basis(2,1)*qt.basis(2,1).dag(),
                 qt.qeye(2),
                 qt.qeye(2))
prob0_2 = (state.dag() * P0_2 * state).tr().real
prob1_2 = (state.dag() * P1_2 * state).tr().real

import numpy as np
if np.random.rand() < prob0_2:
    state = (P0_2*state) / np.sqrt(prob0_2)
    measure_qubit2 = 0
else:
    state = (P1_2*state) / np.sqrt(prob1_2)
    measure_qubit2 = 1

# Trace out qubit2 -> keep [0,2,3] i.e. qubits 1,3,4
state = state.ptrace([0,2,3])  # dims now: [2,2,2]

# 6) If measurement_qubit2 == 1, apply X to qubit4
#    Now qubit4 is index=2 in [1,3,4].
if measure_qubit2 == 1:
    X4 = qt.tensor(qt.qeye(2), qt.qeye(2), qt.sigmax())
    state = X4 * state * X4.dag()

# 7) Measure qubit3 in X basis & trace out
#    Qubit3 is index=1 in [1,3,4].
#    We'll do manual hadamard again, then measure in Z.
H3 = qt.tensor(qt.qeye(2), manual_hadamard_qobj(), qt.qeye(2))
state = H3 * state * H3.dag()

P0_3 = qt.tensor(qt.qeye(2),
                 qt.basis(2,0)*qt.basis(2,0).dag(),
                 qt.qeye(2))
P1_3 = qt.tensor(qt.qeye(2),
                 qt.basis(2,1)*qt.basis(2,1).dag(),
                 qt.qeye(2))
prob0_3 = (state.dag()*P0_3*state).tr().real
prob1_3 = (state.dag()*P1_3*state).tr().real

if np.random.rand() < prob0_3:
    state = (P0_3*state)/np.sqrt(prob0_3)
    measure_qubit3 = 0
else:
    state = (P1_3*state)/np.sqrt(prob1_3)
    measure_qubit3 = 1

# Trace out qubit3 -> keep [0,2] i.e. qubits 1,4
state = state.ptrace([0,2])  # dims now: [2,2]

# 8) If measure_qubit3 == 1, apply Z to qubit1
if measure_qubit3 == 1:
    Z1 = qt.tensor(qt.sigmaz(), qt.qeye(2))
    state = Z1 * state * Z1.dag()

print("Final two-qubit state (qubits 1,4):\n", state)
print("Measurement of qubit2 (Z-basis):", measure_qubit2)
print("Measurement of qubit3 (X-basis):", measure_qubit3)


ValueError: Targets must be smaller than N=4.

In [3]:
import qutip as qt
import numpy as np

def quantum_circuit():
    """
    Implements a 4-qubit quantum circuit with measurement-dependent corrections.
    
    Returns:
    - Final reduced density matrix after all operations.
    - Measurement results of qubits 2 and 3.
    """
    # Step 1: Initialize 4 qubits in |0000⟩
    state = qt.tensor(qt.basis(2, 0), qt.basis(2, 0), qt.basis(2, 0), qt.basis(2, 0))  # |0000⟩

    # Step 2: Create Bell state between qubit 2 and qubit 3
    H2 = qt.tensor(qt.qeye(2), qt.qeye(2), qt.hadamard_transform(), qt.qeye(2))  # Hadamard on qubit 2
    CNOT_23 = qt.cnot(4, control=2, target=3)  # CNOT on qubits 2 → 3
    state = CNOT_23 * H2 * state  # Apply Hadamard & CNOT to create Bell state

    # Step 3: Apply CNOT gate (qubit 1 → qubit 2)
    CNOT_12 = qt.cnot(4, control=1, target=2)
    state = CNOT_12 * state

    # Step 4: Apply CNOT gate (qubit 3 → qubit 4)
    CNOT_34 = qt.cnot(4, control=3, target=4)
    state = CNOT_34 * state

    # Step 5: Measure qubit 2 in the Z basis and trace it out
    P0 = qt.tensor(qt.qeye(2), qt.basis(2, 0) * qt.basis(2, 0).dag(), qt.qeye(2), qt.qeye(2))  # Projection |0⟩⟨0|
    P1 = qt.tensor(qt.qeye(2), qt.basis(2, 1) * qt.basis(2, 1).dag(), qt.qeye(2), qt.qeye(2))  # Projection |1⟩⟨1|
    
    prob_0 = (state.dag() * P0 * state).tr().real  # Probability of |0⟩
    prob_1 = (state.dag() * P1 * state).tr().real  # Probability of |1⟩

    # Collapse the state based on measurement outcome
    if np.random.rand() < prob_0:
        state = (P0 * state) / np.sqrt(prob_0)
        measurement_2 = 0
    else:
        state = (P1 * state) / np.sqrt(prob_1)
        measurement_2 = 1

    # Trace out qubit 2
    state = state.ptrace([0, 1, 3])  # Keep qubits 1, 3, 4 (remove qubit 2)

    # Step 6: Apply X to qubit 4 if measurement_2 == 1
    if measurement_2 == 1:
        X4 = qt.tensor(qt.qeye(2), qt.qeye(2), qt.sigmax())  # X gate on qubit 4
        state = X4 * state * X4.dag()

    # Step 7: Measure qubit 3 in the X basis and trace it out
    H3 = qt.tensor(qt.qeye(2), qt.hadamard_transform(), qt.qeye(2))  # Hadamard on qubit 3
    state = H3 * state * H3.dag()  # Rotate basis

    P0_x = qt.tensor(qt.qeye(2), qt.basis(2, 0) * qt.basis(2, 0).dag(), qt.qeye(2))  # Projection |+⟩⟨+|
    P1_x = qt.tensor(qt.qeye(2), qt.basis(2, 1) * qt.basis(2, 1).dag(), qt.qeye(2))  # Projection |−⟩⟨−|

    prob_0_x = (state.dag() * P0_x * state).tr().real
    prob_1_x = (state.dag() * P1_x * state).tr().real

    # Collapse state based on X-basis measurement
    if np.random.rand() < prob_0_x:
        state = (P0_x * state) / np.sqrt(prob_0_x)
        measurement_3 = 0
    else:
        state = (P1_x * state) / np.sqrt(prob_1_x)
        measurement_3 = 1

    # Trace out qubit 3
    state = state.ptrace([0, 2])  # Keep qubits 1, 4 (remove qubit 3)

    # Step 8: Apply Z to qubit 1 if measurement_3 == 1
    if measurement_3 == 1:
        Z1 = qt.tensor(qt.sigmaz(), qt.qeye(2))  # Z gate on qubit 1
        state = Z1 * state * Z1.dag()

    return state, measurement_2, measurement_3

# Run the quantum circuit
final_state, m2, m3 = quantum_circuit()

# Print final state and measurement results
print("Final reduced density matrix (qubits 1 and 4):")
print(final_state)

print("\nMeasurement of qubit 2 (Z-basis):", m2)
print("Measurement of qubit 3 (X-basis):", m3)


AttributeError: module 'qutip' has no attribute 'hadamard_transform'